In [ ]:
# | default_exp application

In [ ]:
# | export

import asyncio
import functools
import json
import tempfile
import time
from asyncio import iscoroutinefunction  # do not use the version from inspect
from contextlib import asynccontextmanager, contextmanager
from copy import deepcopy
from datetime import datetime, timedelta
from enum import Enum
from inspect import signature
from os import environ
from pathlib import Path
from typing import *
from typing import get_type_hints

import anyio 
import asyncer
import confluent_kafka
import httpx
import yaml
from aiokafka import AIOKafkaConsumer, AIOKafkaProducer
from confluent_kafka import KafkaError, Message, Producer
from confluent_kafka.admin import AdminClient, NewTopic
from fastapi import Depends, FastAPI, HTTPException, Request, Response, status
from fastapi.openapi.docs import get_redoc_html, get_swagger_ui_html
from fastapi.openapi.utils import get_openapi
from fastapi.responses import FileResponse, RedirectResponse
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from fastapi.staticfiles import StaticFiles
from fastcore.foundation import patch
from fastcore.meta import delegates
from pydantic import BaseModel, EmailStr, Field, HttpUrl, PositiveInt
from pydantic.json import timedelta_isoformat
from pydantic.schema import schema

import fast_kafka_api._components.logger

fast_kafka_api._components.logger.should_supress_timestamps = True

import fast_kafka_api
from fast_kafka_api._components.aiokafka_consumer_loop import aiokafka_consumer_loop, sanitize_kafka_config
from fast_kafka_api._components.aiokafka_producer_manager import AIOKafkaProducerManager
from fast_kafka_api._components.asyncapi import (
    ConsumeCallable,
    ContactInfo,
    KafkaBroker,
    KafkaBrokers,
    KafkaMessage,
    KafkaServiceInfo,
    ProduceCallable,
    export_async_spec,
)
from fast_kafka_api._components.logger import get_logger, supress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
import shutil
import unittest.mock
from dataclasses import dataclass

import nest_asyncio
import pytest
import uvicorn
import yaml
from fastapi.testclient import TestClient
from rich.pretty import pprint
from starlette.datastructures import Headers

from fast_kafka_api.testing import mock_AIOKafkaProducer_send, true_after

In [ ]:
# | notest

# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

kafka_config = {
    "bootstrap_servers": f"{kafka_server_url}:{kafka_server_port}",
    "group_id": f"{kafka_server_url}:{kafka_server_port}_group",
    "auto_offset_reset": "earliest",
}

### Constructor utilities

In [ ]:
# | export


@delegates(FastAPI) # type: ignore
def _get_fast_api_app(
    fast_api_app: Optional[FastAPI] = None,
    **kwargs: Dict[str, Any],
) -> FastAPI:
    if fast_api_app is None:
        return FastAPI(
            **kwargs # type: ignore
        )
    else:
        return fast_api_app

In [ ]:
app = _get_fast_api_app()
assert isinstance(app, FastAPI)
print("\n".join([f"{k}={v}" for k, v in app.__dict__.items()]))

_debug=False
title=FastAPI
description=
version=0.1.0
terms_of_service=None
contact=None
license_info=None
openapi_url=/openapi.json
openapi_tags=None
root_path_in_servers=True
docs_url=/docs
redoc_url=/redoc
swagger_ui_oauth2_redirect_url=/docs/oauth2-redirect
swagger_ui_init_oauth=None
swagger_ui_parameters=None
servers=[]
extra={}
openapi_version=3.0.2
openapi_schema=None
root_path=
state=<starlette.datastructures.State object>
dependency_overrides={}
router=<fastapi.routing.APIRouter object>
exception_handlers={<class 'starlette.exceptions.HTTPException'>: <function http_exception_handler>, <class 'fastapi.exceptions.RequestValidationError'>: <function request_validation_exception_handler>}
user_middleware=[]
middleware_stack=<starlette.middleware.errors.ServerErrorMiddleware object>


In [ ]:
# |export

def _get_func_with_combined_sig(
    fx: List[Callable[[Any], Any]], keep: bool = False
) -> Callable[[Any], Any]:
    """Creates a no-op function with a signature containing parameters from signatures of all functions from `fx`
    
    Args:
        fx: a list of functions
        keep: if **True**, keep kwargs as an argument
        
    Returns:
        no-op function with a signature containing parameters from signatures of all functions from `fx`
    """
    def _f(**kwargs):
        pass

    retval = _f
    for f in fx:
        retval = delegates(f, keep=True)(retval)
    if not keep:
        retval = delegates(_f, keep=False)(retval)
    return retval # type: ignore

In [ ]:
f = _get_func_with_combined_sig([AIOKafkaConsumer.__init__, AIOKafkaProducer.__init__])
assert len(signature(f).parameters.keys()) == 48

In [ ]:
f = _get_func_with_combined_sig([AIOKafkaConsumer.__init__, AIOKafkaProducer.__init__], keep=True)
assert len(signature(f).parameters.keys()) == 49

In [ ]:
# | export


@delegates( # type: ignore
    _get_func_with_combined_sig([AIOKafkaConsumer.__init__, AIOKafkaProducer.__init__])
)
def _get_kafka_config(
    **kwargs,
) -> Dict[str, Any]:
    allowed_keys = set(signature(_get_kafka_config).parameters.keys())
    if not set(kwargs.keys()) <= allowed_keys:
        unallowed_keys = ", ".join(
            sorted([f"'{x}'" for x in set(kwargs.keys()).difference(allowed_keys)])
        )
        raise ValueError(f"Unallowed key arguments passed: {unallowed_keys}")
    retval = kwargs.copy()

    # todo: check this values
    config_defaults = {
        "bootstrap_servers": "localhost:9092",
        "auto_offset_reset": "earliest",
        "max_poll_records": 100,
        #         "max_buffer_size": 10_000,
    }
    for key, value in config_defaults.items():
        if key not in retval:
            retval[key] = value

    return retval

In [ ]:
assert _get_kafka_config() == {
    "bootstrap_servers": "localhost:9092",
    "auto_offset_reset": "earliest",
    "max_poll_records": 100,
}

assert _get_kafka_config(max_poll_records=1_000) == {
    "bootstrap_servers": "localhost:9092",
    "auto_offset_reset": "earliest",
    "max_poll_records": 1_000,
}

In [ ]:
with pytest.raises(ValueError) as e:
    _get_kafka_config(random_key=1_000, whatever="whocares")
assert e.value.args == ("Unallowed key arguments passed: 'random_key', 'whatever'",)

In [ ]:
# | export


def _get_kafka_brokers(
    kafka_brokers: Optional[Dict[str, Any]] = None
) -> KafkaBrokers:
    if kafka_brokers is None:
        retval:KafkaBrokers = KafkaBrokers(brokers={
            "localhost": KafkaBroker(
                url="https://localhost",
                description="Local (dev) Kafka broker",
                port="9092",
            )
        })
    else:
        retval = KafkaBrokers(brokers={
            k: KafkaBroker.parse_raw(v.json() if hasattr(v, "json") else json.dumps(v))
            for k, v in kafka_brokers.items()
        })
        
    return retval

In [ ]:
assert (
    _get_kafka_brokers(None).json()
    == '{"brokers": {"localhost": {"url": "https://localhost", "description": "Local (dev) Kafka broker", "protocol": "kafka", "variables": {"port": {"default": "9092"}}}}}'
)

assert (
    _get_kafka_brokers(dict(localhost=dict(url="localhost"))).json()
    == '{"brokers": {"localhost": {"url": "localhost", "description": "Kafka broker", "protocol": "kafka", "variables": {"port": {"default": "9092"}}}}}'
)

assert (
    _get_kafka_brokers(
        dict(localhost=dict(url="localhost"), staging=dict(url="staging.airt.ai"))
    ).json()
    == '{"brokers": {"localhost": {"url": "localhost", "description": "Kafka broker", "protocol": "kafka", "variables": {"port": {"default": "9092"}}}, "staging": {"url": "staging.airt.ai", "description": "Kafka broker", "protocol": "kafka", "variables": {"port": {"default": "9092"}}}}}'
)

In [ ]:
# | export


def _get_topic_name(
    topic_callable: Union[ConsumeCallable, ProduceCallable], prefix: str = "on_"
) -> str:
    topic = topic_callable.__name__
    if not topic.startswith(prefix) or len(topic) <= len(prefix):
        raise ValueError(f"Function name '{topic}' must start with {prefix}")
    topic = topic[len(prefix) :]

    return topic

In [ ]:
def on_topic_name_1():
    pass


assert _get_topic_name(on_topic_name_1) == "topic_name_1"

assert _get_topic_name(on_topic_name_1, prefix="on_topic_") == "name_1"

In [ ]:
#| export 

def _get_contact_info(
    name: str="Author", url: str="https://www.google.com", email:str="noreply@gmail.com"
) -> ContactInfo:
    return ContactInfo(name=name, url=url, email=email)

In [ ]:
_get_contact_info()

ContactInfo(name='Author', url=HttpUrl('https://www.google.com', ), email='noreply@gmail.com')

In [ ]:
# | export


class FastKafkaAPI:
    @delegates(  # type: ignore
        _get_func_with_combined_sig(
            [AIOKafkaConsumer.__init__, AIOKafkaProducer.__init__]
        )
    )
    def __init__(
        self,
        fast_api_app: FastAPI,
        *,
        asyncapi_route: Optional[str] = "/asyncapi",
        title: Optional[str] = None,
        description: Optional[str] = None,
        version: Optional[str] = None,
        contact: Optional[Dict[str, str]] = None,
        kafka_brokers: Optional[Dict[str, Any]] = None,
        root_path: Optional[Union[Path, str]] = None,
        **kwargs,
    ):
        """Combined REST and Kafka service

        Params:
            fast_api_app: the FastAPI app, if None, one will be created
            asyncapi_route: the route to where to mount the documentation. If **None**, the docs will not be mounted.
            title: optional title for the documentation. If None, the title of passed fast_api_app will be used
            description: optional description for the documentation. If None, the description of passed fast_api_app will be used
            version: optional version for the documentation. If None, the version of passed fast_api_app will be used
            contact: optional contact for the documentation. If None, the contact of passed fast_api_app will be used
            kafka_brokers: dictionary describing kafka brokers used for generating documentation
            root_path: path to where documentation will be created
        """
        self._fast_api_app = fast_api_app

        # this is neede for documentation generation
        self._title = title if title else fast_api_app.title
        self._description = description if description else fast_api_app.description
        self._version = version if version else fast_api_app.version
        if contact is not None:
            self._contact_info = _get_contact_info(**contact)
        elif fast_api_app.contact is not None:
            if isinstance(fast_api_app.contact, str):
                self._contact_info = _get_contact_info(name=fast_api_app.contact)
            else:
                self._contact_info = _get_contact_info(**fast_api_app.contact)
        else:
            self._contact_info = _get_contact_info()

        self._kafka_service_info = KafkaServiceInfo(
            title=self._title,
            version=self._version,
            description=self._description,
            contact=self._contact_info,
        )
        self._kafka_brokers = _get_kafka_brokers(kafka_brokers)

        self._root_path = Path(".") if root_path is None else Path(root_path)

        # this is used as default parameters for creating AIOProducer and AIOConsumer objects
        self._kafka_config = _get_kafka_config(**kwargs)

        #
        self._consumers_store: Dict[str, Tuple[ConsumeCallable, Dict[str, Any]]] = {}

        self._producers_list: List[  # type: ignore
            Union[AIOKafkaProducer, AIOKafkaProducerManager]
        ] = []
        self._producers_store: Dict[  # type: ignore
            str, Tuple[ProduceCallable, AIOKafkaProducer, Dict[str, Any]]
        ] = {}

        # background tasks
        self._scheduled_bg_tasks: List[Callable[..., Coroutine[Any, Any, Any]]] = []
        self._bg_task_group_generator: Optional[anyio.abc.TaskGroup] = None
        self._bg_tasks_group: Optional[anyio.abc.TaskGroup]

        # todo: use this for errrors
        self._on_error_topic: Optional[str] = None

        self._asyncapi_path = self._root_path / "asyncapi"
        (self._asyncapi_path / "docs").mkdir(exist_ok=True, parents=True)
        (self._asyncapi_path / "spec").mkdir(exist_ok=True, parents=True)
        self._fast_api_app.mount(
            "/asyncapi",
            StaticFiles(directory=self._asyncapi_path / "docs"),
            name="asyncapi",
        )

        self._is_shutting_down: bool = False
        self._kafka_consumer_tasks: List[asyncio.Task[Any]] = []
        self._kafka_producer_tasks: List[asyncio.Task[Any]] = []

        @self._fast_api_app.get("/", include_in_schema=False)
        def redirect_root_to_asyncapi():
            return RedirectResponse("/asyncapi")

        @self._fast_api_app.get("/asyncapi", include_in_schema=False)
        async def redirect_asyncapi_docs():
            return RedirectResponse("/asyncapi/index.html")

        @self._fast_api_app.get("/asyncapi.yml", include_in_schema=False)
        async def download_asyncapi_yml():
            return FileResponse(self._asyncapi_path / "spec" / "asyncapi.yml")

        @self._fast_api_app.on_event("startup")
        async def on_startup(app=self):
            await app._on_startup()

        @self._fast_api_app.on_event("shutdown")
        async def on_shutdown(app=self):
            await app._on_shutdown()

    async def _on_startup(self) -> None:
        raise NotImplementedError

    async def _on_shutdown(self) -> None:
        raise NotImplementedError

    def consumes(
        self,
        topic: Optional[str] = None,
        *,
        prefix: str = "on_",
        **kwargs: Dict[str, Any],
    ) -> ConsumeCallable:
        raise NotImplementedError

    def produces(  # type: ignore
        self,
        topic: Optional[str] = None,
        *,
        prefix: str = "to_",
        producer: Optional[AIOKafkaProducer] = None,
        **kwargs: Dict[str, Any],
    ) -> ProduceCallable:
        raise NotImplementedError

    def run_in_background(
        self,
    ) -> Callable[[], Any]:
        raise NotImplementedError

    def _populate_consumers(
        self,
        is_shutting_down_f: Callable[[], bool],
    ) -> None:
        raise NotImplementedError

    async def _populate_producers(self) -> None:
        raise NotImplementedError

    def generate_async_spec(self) -> None:
        raise NotImplementedError

    async def _shutdown_consumers(self) -> None:
        raise NotImplementedError

    async def _shutdown_producers(self) -> None:
        raise NotImplementedError

    async def _populate_bg_tasks(self) -> None:
        raise NotImplementedError

    async def _shutdown_bg_tasks(self) -> None:
        raise NotImplementedError

In [ ]:
app = FastAPI()
kafka_app = FastKafkaAPI(app)
kafka_app.__dict__

{'_fast_api_app': <fastapi.applications.FastAPI>,
 '_title': 'FastAPI',
 '_description': '',
 '_version': '0.1.0',
 '_contact_info': ContactInfo(name='Author', url=HttpUrl('https://www.google.com', ), email='noreply@gmail.com'),
 '_kafka_service_info': KafkaServiceInfo(title='FastAPI', version='0.1.0', description='', contact=ContactInfo(name='Author', url=HttpUrl('https://www.google.com', ), email='noreply@gmail.com')),
 '_kafka_brokers': KafkaBrokers(brokers={'localhost': KafkaBroker(url='https://localhost', description='Local (dev) Kafka broker', port='9092', protocol='kafka', security=None)}),
 '_root_path': PosixPath('.'),
 '_kafka_config': {'bootstrap_servers': 'localhost:9092',
  'auto_offset_reset': 'earliest',
  'max_poll_records': 100},
 '_consumers_store': {},
 '_producers_list': [],
 '_producers_store': {},
 '_scheduled_bg_tasks': [],
 '_bg_task_group_generator': None,
 '_on_error_topic': None,
 '_asyncapi_path': PosixPath('asyncapi'),
 '_is_shutting_down': False,
 '_kafka_

In [ ]:
app = FastAPI(contact="Davor")
kafka_app = FastKafkaAPI(app)
kafka_app.__dict__

{'_fast_api_app': <fastapi.applications.FastAPI>,
 '_title': 'FastAPI',
 '_description': '',
 '_version': '0.1.0',
 '_contact_info': ContactInfo(name='Davor', url=HttpUrl('https://www.google.com', ), email='noreply@gmail.com'),
 '_kafka_service_info': KafkaServiceInfo(title='FastAPI', version='0.1.0', description='', contact=ContactInfo(name='Davor', url=HttpUrl('https://www.google.com', ), email='noreply@gmail.com')),
 '_kafka_brokers': KafkaBrokers(brokers={'localhost': KafkaBroker(url='https://localhost', description='Local (dev) Kafka broker', port='9092', protocol='kafka', security=None)}),
 '_root_path': PosixPath('.'),
 '_kafka_config': {'bootstrap_servers': 'localhost:9092',
  'auto_offset_reset': 'earliest',
  'max_poll_records': 100},
 '_consumers_store': {},
 '_producers_list': [],
 '_producers_store': {},
 '_scheduled_bg_tasks': [],
 '_bg_task_group_generator': None,
 '_on_error_topic': None,
 '_asyncapi_path': PosixPath('asyncapi'),
 '_is_shutting_down': False,
 '_kafka_co

In [ ]:
def create_testing_app():
    root_path = "/tmp/000_FastKafkaAPI"
    if Path(root_path).exists():
        shutil.rmtree(root_path)

    app = FastAPI()
    kafka_app = FastKafkaAPI(
        app,
        kafka_brokers={
            "local": {
                "url": "kafka",
                "name": "development",
                "description": "Local (dev) Kafka broker",
                "port": 9092,
            }
        },
        root_path=root_path,
        **kafka_config,
    )

    return kafka_app

In [ ]:
app = create_testing_app()
app

<__main__.FastKafkaAPI>

In [ ]:
# | export


@patch  # type: ignore
def consumes(
    self: FastKafkaAPI,
    topic: Optional[str] = None,
    *,
    prefix: str = "on_",
    **kwargs: Dict[str, Any],
) -> Callable[[ConsumeCallable], ConsumeCallable]:
    """Decorator registering the callback called when a message is received in a topic.

    This function decorator is also responsible for registering topics for AsyncAPI specificiation and documentation.

    Params:
        topic: Kafka topic that the consumer will subscribe to and execute the decorated function when it receives a message from the topic, default: None
            If the topic is not specified, topic name will be inferred from the decorated function name by stripping the defined prefix
        prefix: Prefix stripped from the decorated function to define a topic name if the topic argument is not passed, default: "on_"
            If the decorated function name is not prefixed with the defined prefix and topic argument is not passed, then this method will throw ValueError
        **kwargs: Keyword arguments that will be passed to AIOKafkaConsumer, used to configure the consumer

    Returns:
        A function returning the same function

    Throws:
        ValueError

    """

    def _decorator(
        on_topic: ConsumeCallable,
        topic: Optional[str] = topic,
        kwargs: Dict[str, Any] = kwargs,
    ) -> ConsumeCallable:
        topic_resolved: str = (
            _get_topic_name(topic_callable=on_topic, prefix=prefix)
            if topic is None
            else topic
        )

        self._consumers_store[topic_resolved] = (on_topic, kwargs)

        return on_topic

    return _decorator

In [ ]:
app = create_testing_app()

# Basic check
@app.consumes()
def on_my_topic_1(msg: BaseModel) -> None:
    pass


assert app._consumers_store["my_topic_1"] == (on_my_topic_1, {}), app._consumers_store

# Check topic setting
@app.consumes(topic="test_topic_2")
def some_func_name(msg: BaseModel) -> None:
    pass


assert app._consumers_store["test_topic_2"] == (
    some_func_name,
    {},
), app._consumers_store

# Check prefix change
@app.consumes(prefix="for_")
def for_test_topic_3(msg: BaseModel) -> None:
    pass


assert app._consumers_store["test_topic_3"] == (
    for_test_topic_3,
    {},
), app._consumers_store

# Check passing of kwargs
kwargs = {"arg1": "val1", "arg2": 2}


@app.consumes(topic="test_topic", **kwargs)
def for_test_kwargs(msg: BaseModel):
    pass


assert app._consumers_store["test_topic"] == (
    for_test_kwargs,
    kwargs,
), app._consumers_store

In [ ]:
# | export


def _to_json_utf8(o: Any) -> bytes:
    """Converts to JSON and then encodes with UTF-8"""
    if hasattr(o, "json"):
        return o.json().encode("utf-8")  # type: ignore
    else:
        return json.dumps(o).encode("utf-8")

In [ ]:
assert _to_json_utf8({"a": 1, "b": [2, 3]}) == b'{"a": 1, "b": [2, 3]}'


class A(BaseModel):
    name: str = Field()
    age: int


assert _to_json_utf8(A(name="Davor", age=12)) == b'{"name": "Davor", "age": 12}'

In [ ]:
# | export


def produce_decorator(
    self: FastKafkaAPI, func: ProduceCallable, topic: str
) -> ProduceCallable:
    @functools.wraps(func)
    async def _produce_async(*args: List[Any], **kwargs: Dict[str, Any]) -> BaseModel:
        f: Callable[..., Awaitable[BaseModel]] = func  # type: ignore
        return_val = await f(*args, **kwargs)
        _, producer, _ = self._producers_store[topic]
        fut = await producer.send(topic, _to_json_utf8(return_val))
        msg = await fut
        return return_val

    @functools.wraps(func)
    def _produce_sync(*args: List[Any], **kwargs: Dict[str, Any]) -> BaseModel:
        f: Callable[..., BaseModel] = func  # type: ignore
        return_val = f(*args, **kwargs)
        _, producer, _ = self._producers_store[topic]
        producer.send(topic, _to_json_utf8(return_val))
        return return_val

    return _produce_async if iscoroutinefunction(func) else _produce_sync  # type: ignore

In [ ]:
async def test_me(is_async: bool):
    with mock_AIOKafkaProducer_send() as send_mock:
        topic = "test_topic"

        class MockMsg(BaseModel):
            name: str = "Micky Mouse"
            id: int = 123

        if is_async:

            async def func(mock_msg: MockMsg) -> MockMsg:
                return mock_msg

        else:

            def func(mock_msg: MockMsg) -> MockMsg:
                return mock_msg

        producer = AIOKafkaProducer(bootstrap_servers=kafka_config["bootstrap_servers"])
        if not is_async:
            producer = AIOKafkaProducerManager(producer)

        await producer.start()
        try:
            app = unittest.mock.Mock()
            app._producers_store = {topic: (func, producer, {})}

            test_func = produce_decorator(app, func, topic)
            assert iscoroutinefunction(test_func) == is_async

            mock_msg = MockMsg()
            if not is_async:
                value = test_func(mock_msg)
                await asyncio.sleep(1)
            else:
                value = await test_func(mock_msg)

            send_mock.assert_called_once_with(topic, mock_msg.json().encode("utf-8"))
            assert value == mock_msg

        finally:
            await producer.stop()


for is_async in [True, False]:
    await test_me(is_async)

print("ok")

[INFO] fast_kafka_api._components.aiokafka_producer_manager: AIOKafkaProducerManager.start(): Entering...
[INFO] fast_kafka_api._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Starting...
[INFO] fast_kafka_api._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Starting task group
[INFO] fast_kafka_api._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Starting send_stream
[INFO] fast_kafka_api._components.aiokafka_producer_manager: AIOKafkaProducerManager.start(): Finished.
[INFO] fast_kafka_api._components.aiokafka_producer_manager: AIOKafkaProducerManager.stop(): Entering...
[INFO] fast_kafka_api._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Exiting send_stream
[INFO] fast_kafka_api._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Exiting task group
[INFO] fast_kafka_api._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Finished.
[INFO] fast_kafka_api._components.aio

In [ ]:
# | export


@patch  # type: ignore
def produces(
    self: FastKafkaAPI,
    topic: Optional[str] = None,
    *,
    prefix: str = "to_",
    producer: AIOKafkaProducer = None,
    **kwargs: Dict[str, Any],
) -> Callable[[ProduceCallable], ProduceCallable]:
    """Decorator registering the callback called when delivery report for a produced message is received

    This function decorator is also responsible for registering topics for AsyncAPI specificiation and documentation.

    Params:
        topic: Kafka topic that the producer will send returned values from the decorated function to, default: None
            If the topic is not specified, topic name will be inferred from the decorated function name by stripping the defined prefix
        prefix: Prefix stripped from the decorated function to define a topic name if the topic argument is not passed, default: "to_"
            If the decorated function name is not prefixed with the defined prefix and topic argument is not passed, then this method will throw ValueError
        producer:
        **kwargs: Keyword arguments that will be passed to AIOKafkaProducer, used to configure the producer

    Returns:
        A function returning the same function

    Throws:
        ValueError

    """

    def _decorator(
        on_topic: ProduceCallable,
        topic: Optional[str] = topic,
        kwargs: Dict[str, Any] = kwargs,
    ) -> ProduceCallable:
        topic_resolved: str = (
            _get_topic_name(topic_callable=on_topic, prefix=prefix)
            if topic is None
            else topic
        )

        self._producers_store[topic_resolved] = (on_topic, producer, kwargs)

        return produce_decorator(self, on_topic, topic_resolved)

    return _decorator

In [ ]:
app = create_testing_app()

# Basic check
async def to_my_topic_1(msg: BaseModel) -> None:
    pass


# Must be done without sugar to keep the original function reference
check_func = to_my_topic_1
to_my_topic_1 = app.produces()(to_my_topic_1)

assert app._producers_store["my_topic_1"] == (
    check_func,
    None,
    {},
), f"{app._producers_store}, {to_my_topic_1}"

# Check topic setting
def some_func_name(msg: BaseModel) -> None:
    pass


check_func = some_func_name
some_func_name = app.produces(topic="test_topic_2")(some_func_name)

assert app._producers_store["test_topic_2"] == (
    check_func,
    None,
    {},
), app._producers_store

# Check prefix change
@app.produces(prefix="for_")
def for_test_topic_3(msg: BaseModel) -> None:
    pass


check_func = for_test_topic_3
some_func_name = app.produces(prefix="for_")(for_test_topic_3)

assert app._producers_store["test_topic_3"] == (
    check_func,
    None,
    {},
), app._producers_store

# Check passing of kwargs
kwargs = {"arg1": "val1", "arg2": 2}


async def for_test_kwargs(msg: BaseModel):
    pass


check_func = for_test_kwargs
for_test_kwargs = app.produces(topic="test_topic", **kwargs)(for_test_kwargs)

assert app._producers_store["test_topic"] == (
    check_func,
    None,
    kwargs,
), app._producers_store

# Check passing of custom Producer
async def test_me():
    kwargs = {"arg1": "val1", "arg2": 2}

    async def for_test_producer(msg: BaseModel):
        pass

    check_func = for_test_producer
    producer = AIOKafkaProducer()
    for_test_producer = app.produces(
        topic="test_producer", producer=producer, **kwargs
    )(for_test_producer)

    assert app._producers_store["test_producer"] == (
        check_func,
        producer,
        kwargs,
    ), app._producers_store


await test_me()

In [ ]:
# | export


@patch # type: ignore
def run_in_background(
    self: FastKafkaAPI,
) -> Callable[[Callable[..., Coroutine[Any, Any, Any]]], Callable[..., Coroutine[Any, Any, Any]]]:
    """
    Decorator to schedule a task to be run in the background.

    This decorator is used to schedule a task to be run in the background when the app's `_on_startup` event is triggered.

    Returns:
        Callable[None, None]: A decorator function that takes a background task as an input and stores it to be run in the backround.
    """
    
    def _decorator(bg_task: Callable[..., Coroutine[Any, Any, Any]]) -> Callable[..., Coroutine[Any, Any, Any]]:
        """
        Store the background task.

        Args:
            bg_task (Callable[[], None]): The background task to be run asynchronously.

        Returns:
            Callable[[], None]: Original background task.
        """
        self._scheduled_bg_tasks.append(bg_task)
        
        return bg_task

    return _decorator

In [ ]:
# Check if the background job is getting registered

app = create_testing_app()

@app.run_in_background()
async def async_background_job():
    pass

assert app._scheduled_bg_tasks[0] == async_background_job, app._scheduled_bg_tasks[0]
assert app._scheduled_bg_tasks.__len__() == 1

In [ ]:
class MyInfo(BaseModel):
    mobile: str = Field(..., example="+385987654321")
    name: str = Field(..., example="James Bond")


class MyMsgUrl(BaseModel):
    info: MyInfo = Field(..., example=dict(mobile="+385987654321", name="James Bond"))
    url: HttpUrl = Field(..., example="https://sis.gov.uk/agents/007")


class MyMsgEmail(BaseModel):
    msg_url: MyMsgUrl = Field(
        ...,
        example=dict(
            info=dict(mobile="+385987654321", name="James Bond"),
            url="https://sis.gov.uk/agents/007",
        ),
    )
    email: EmailStr = Field(..., example="agent-007@sis.gov.uk")


def setup_testing_app():
    app = create_testing_app()

    @app.consumes("my_topic_1")
    def on_my_topic_one(msg: MyMsgUrl) -> None:
        logger.debug(f"on_my_topic_one(msg={msg},)")

    @app.consumes()
    async def on_my_topic_2(msg: MyMsgEmail) -> None:
        logger.debug(f"on_my_topic_2(msg={msg},)")

    with pytest.raises(ValueError) as e:

        @app.consumes()
        def my_topic_3(msg: MyMsgEmail) -> None:
            raise NotImplemented

    @app.produces()
    def to_my_topic_3(url: str) -> MyMsgUrl:
        logger.debug(f"on_my_topic_3(msg={url}")
        return MyMsgUrl(info=MyInfo("+3851987654321", "Sean Connery"), url=url)

    @app.produces()
    async def to_my_topic_4(msg: MyMsgEmail) -> MyMsgEmail:
        logger.debug(f"on_my_topic_4(msg={msg}")
        return msg

    @app.produces()
    def to_my_topic_5(url: str) -> MyMsgUrl:
        logger.debug(f"on_my_topic_5(msg={url}")
        return MyMsgUrl(info=MyInfo("+3859123456789", "John Wayne"), url=url)
    
    @app.run_in_background()
    async def long_bg_job():
        await asyncio.sleep(100) 

    return app

In [ ]:
app = setup_testing_app()

assert set(app._consumers_store.keys()) == set(["my_topic_1", "my_topic_2"])
assert set(app._producers_store.keys()) == set(
    ["my_topic_3", "my_topic_4", "my_topic_5"]
)

print(f"app._kafka_service_info={app._kafka_service_info}")
print(f"app._kafka_brokers={app._kafka_brokers}")

app._kafka_service_info=title='FastAPI' version='0.1.0' description='' contact=ContactInfo(name='Author', url=HttpUrl('https://www.google.com', ), email='noreply@gmail.com')
app._kafka_brokers=brokers={'local': KafkaBroker(url='kafka', description='Local (dev) Kafka broker', port='9092', protocol='kafka', security=None)}


In [ ]:
# | export


def filter_using_signature(f: Callable, **kwargs: Dict[str, Any]) -> Dict[str, Any]:
    param_names = list(signature(f).parameters.keys())
    return {k: v for k, v in kwargs.items() if k in param_names}

In [ ]:
def f(a: int, *, b: str):
    pass


assert filter_using_signature(f, a=1, c=3) == {"a": 1}

In [ ]:
# | export


@patch  # type: ignore
def _populate_consumers(
    self: FastKafkaAPI,
    is_shutting_down_f: Callable[[], bool],
) -> None:
    default_config: Dict[str, Any] = filter_using_signature(
        AIOKafkaConsumer, **self._kafka_config
    )
    self._kafka_consumer_tasks = [
        asyncio.create_task(
            aiokafka_consumer_loop(
                topics=[topic],
                callbacks={topic: consumer},
                msg_types={topic: signature(consumer).parameters["msg"].annotation},
                is_shutting_down_f=is_shutting_down_f,
                **{**default_config, **override_config},
            )
        )
        for topic, (consumer, override_config) in self._consumers_store.items()
    ]


@patch  # type: ignore
async def _shutdown_consumers(
    self: FastKafkaAPI,
) -> None:
    if self._kafka_consumer_tasks:
        await asyncio.wait(self._kafka_consumer_tasks)

In [ ]:
app = setup_testing_app()
app._populate_consumers(is_shutting_down_f=true_after(1))
assert len(app._kafka_consumer_tasks) == 2

await app._shutdown_consumers()

assert all([t.done() for t in app._kafka_consumer_tasks])

[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': 'tvrtko-fast-kafka-api-kafka-1:9092', 'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'group_id': 'tvrtko-fast-kafka-api-kafka-1:9092_group'}
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': 'tvrtko-fast-kafka-api-kafka-1:9092', 'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'group_id': 'tvrtko-fast-kafka-api-kafka-1:9092_group'}
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'my_top

In [ ]:
# | export

# TODO: Add passing of vars
async def _create_producer(  # type: ignore
    *,
    callback: ProduceCallable,
    producer: Optional[AIOKafkaProducer],
    default_config: Dict[str, Any],
    override_config: Dict[str, Any],
    producers_list: List[Union[AIOKafkaProducer, AIOKafkaProducerManager]],
) -> Union[AIOKafkaProducer, AIOKafkaProducerManager]:
    """Creates a producer

    Args:
        callback: A callback function that is called when the producer is ready.
        producer: An existing producer to use.
        default_config: A dictionary of default configuration values.
        override_config: A dictionary of configuration values to override.
        producers_list: A list of producers to add the new producer to.

    Returns:
        A producer.
    """

    if producer is None:
        config = {
            **filter_using_signature(AIOKafkaProducer, **default_config),
            **override_config,
        }
        producer = AIOKafkaProducer(**config)
        logger.info(f"_create_producer() : created producer using the config: '{sanitize_kafka_config(**config)}'")

    if not iscoroutinefunction(callback):
        producer = AIOKafkaProducerManager(producer)

    await producer.start()

    producers_list.append(producer)

    return producer


@patch  # type: ignore
async def _populate_producers(self: FastKafkaAPI) -> None:
    """Populates the producers for the FastKafkaAPI instance.

    Args:
        self: The FastKafkaAPI instance.

    Returns:
        None.

    Raises:
        None.
    """
    default_config: Dict[str, Any] = self._kafka_config
    self._producers_list = []
    self._producers_store = {
        topic: (
            callback,
            await _create_producer(
                callback=callback,
                producer=producer,
                default_config=default_config,
                override_config=override_config,
                producers_list=self._producers_list,
            ),
            override_config,
        )
        for topic, (
            callback,
            producer,
            override_config,
        ) in self._producers_store.items()
    }


@patch  # type: ignore
async def _shutdown_producers(self: FastKafkaAPI) -> None:
    [await producer.stop() for producer in self._producers_list[::-1]]

In [ ]:
app = setup_testing_app()
await app._populate_producers()
await app._shutdown_producers()
assert len(app._producers_list) == 3

app._producers_list

[INFO] __main__: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fast-kafka-api-kafka-1:9092'}'
[INFO] fast_kafka_api._components.aiokafka_producer_manager: AIOKafkaProducerManager.start(): Entering...
[INFO] fast_kafka_api._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Starting...
[INFO] fast_kafka_api._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Starting task group
[INFO] fast_kafka_api._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Starting send_stream
[INFO] fast_kafka_api._components.aiokafka_producer_manager: AIOKafkaProducerManager.start(): Finished.
[INFO] __main__: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fast-kafka-api-kafka-1:9092'}'
[INFO] __main__: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fast-kafka-api-kafka-1:9092'}'
[INFO] fast_kafka_api._components.aiokafka_producer_manager

[<fast_kafka_api._components.aiokafka_producer_manager.AIOKafkaProducerManager>,
 <aiokafka.producer.producer.AIOKafkaProducer>,
 <fast_kafka_api._components.aiokafka_producer_manager.AIOKafkaProducerManager>]

In [ ]:
# | export


@patch  # type: ignore
async def _populate_bg_tasks(
    self: FastKafkaAPI,
) -> None:
    self._bg_task_group_generator = anyio.create_task_group()
    self._bg_tasks_group = await self._bg_task_group_generator.__aenter__()
    for task in self._scheduled_bg_tasks:
        self._bg_tasks_group.start_soon(task)


@patch  # type: ignore
async def _shutdown_bg_tasks(
    self: FastKafkaAPI,
) -> None:
    self._bg_tasks_group.cancel_scope.cancel()  # type: ignore
    await self._bg_task_group_generator.__aexit__(None, None, None)  # type: ignore

In [ ]:
app = setup_testing_app()
await app._populate_bg_tasks()
assert len(app._scheduled_bg_tasks) == 1
assert app._bg_task_group_generator is not None
assert app._bg_tasks_group is not None

await app._shutdown_bg_tasks()

In [ ]:
# | export


@patch  # type: ignore
def generate_async_spec(self: FastKafkaAPI) -> None:
    export_async_spec(
        consumers={
            topic: callback for topic, (callback, _) in self._consumers_store.items()
        },
        producers={
            topic: callback for topic, (callback, _, _) in self._producers_store.items()
        },
        kafka_brokers=self._kafka_brokers,
        kafka_service_info=self._kafka_service_info,
        asyncapi_path=self._asyncapi_path,
    )

In [ ]:
app = setup_testing_app()
app.generate_async_spec()

[INFO] fast_kafka_api._components.asyncapi: Old async specifications at '/tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fast_kafka_api._components.asyncapi: New async specifications generated at: '/tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml'
[INFO] fast_kafka_api._components.asyncapi: Async docs generated at '/tmp/000_FastKafkaAPI/asyncapi/docs'
[INFO] fast_kafka_api._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag /tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml @asyncapi/html-template -o /tmp/000_FastKafkaAPI/asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/000_FastKafkaAPI/asyncapi/docs.




In [ ]:
# | export


@patch  # type: ignore
async def _on_startup(self: FastKafkaAPI) -> None:

    self._is_shutting_down = False

    def is_shutting_down_f(self: FastKafkaAPI = self) -> bool:
        return self._is_shutting_down

    self.generate_async_spec()
    await self._populate_producers()
    self._populate_consumers(is_shutting_down_f)
    await self._populate_bg_tasks()


@patch  # type: ignore
async def _on_shutdown(self: FastKafkaAPI) -> None:
    self._is_shutting_down = True

    await self._shutdown_bg_tasks()
    await self._shutdown_consumers()
    await self._shutdown_producers()

In [ ]:
@asynccontextmanager
async def start_test_app():
    app = setup_testing_app()
    try:
        await app._on_startup()
        yield app
    finally:
        await app._on_shutdown()

In [ ]:
expected = """asyncapi: 2.5.0
info:
  description: ''
  title: FastAPI
  version: 0.1.0
  contact:
    email: noreply@gmail.com
    name: Author
    url: https://www.google.com
servers:
  local:
    description: Local (dev) Kafka broker
    protocol: kafka
    url: kafka
    variables:
      port:
        default: '9092'
channels:
  my_topic_1:
    subscribe:
      message:
        $ref: '#/components/messages/MyMsgUrl'
  my_topic_2:
    subscribe:
      message:
        $ref: '#/components/messages/MyMsgEmail'
  my_topic_3:
    publish:
      message:
        $ref: '#/components/messages/MyMsgUrl'
  my_topic_4:
    publish:
      message:
        $ref: '#/components/messages/MyMsgEmail'
  my_topic_5:
    publish:
      message:
        $ref: '#/components/messages/MyMsgUrl'
components:
  messages:
    MyMsgEmail:
      payload:
        example:
          email: agent-007@sis.gov.uk
          msg_url:
            info:
              mobile: '+385987654321'
              name: James Bond
            url: https://sis.gov.uk/agents/007
        properties:
          email:
            example: agent-007@sis.gov.uk
            format: email
            title: Email
            type: string
          msg_url:
            allOf:
            - $ref: '#/components/messages/MyMsgUrl'
            example:
              info:
                mobile: '+385987654321'
                name: James Bond
              url: https://sis.gov.uk/agents/007
            title: Msg Url
        required:
        - msg_url
        - email
        title: MyMsgEmail
        type: object
    MyMsgUrl:
      payload:
        example:
          info:
            mobile: '+385987654321'
            name: James Bond
          url: https://sis.gov.uk/agents/007
        properties:
          info:
            allOf:
            - $ref: '#/components/schemas/MyInfo'
            example:
              mobile: '+385987654321'
              name: James Bond
            title: Info
          url:
            example: https://sis.gov.uk/agents/007
            format: uri
            maxLength: 2083
            minLength: 1
            title: Url
            type: string
        required:
        - info
        - url
        title: MyMsgUrl
        type: object
  schemas:
    MyInfo:
      payload:
        properties:
          mobile:
            example: '+385987654321'
            title: Mobile
            type: string
          name:
            example: James Bond
            title: Name
            type: string
        required:
        - mobile
        - name
        title: MyInfo
        type: object
  securitySchemes: {}
"""

In [ ]:
d1, d2 = None, None


async def test_me():
    global d1
    global d2
    async with start_test_app() as app:
        client = TestClient(app._fast_api_app)
        response = client.get("/asyncapi.yml")
        assert response.status_code == 200
        d1 = yaml.safe_load(response.text)
        d2 = yaml.safe_load(expected)
        assert d1 == d2, f"{d1} != {d2}"


asyncio.run(test_me())
print("ok")

[INFO] fast_kafka_api._components.asyncapi: Old async specifications at '/tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fast_kafka_api._components.asyncapi: New async specifications generated at: '/tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml'
[INFO] fast_kafka_api._components.asyncapi: Async docs generated at '/tmp/000_FastKafkaAPI/asyncapi/docs'
[INFO] fast_kafka_api._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag /tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml @asyncapi/html-template -o /tmp/000_FastKafkaAPI/asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/000_FastKafkaAPI/asyncapi/docs.


[INFO] __main__: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fast-kafka-api-kafka-1:9092'}'
[INFO] fast_kafka_api._components.aiokafka_producer_manager: AIOKafkaProducerManager.start(): Entering...
[INFO] fast_kafka_api._components.aiokafka_producer_manager: _aiokafka_

In [ ]:
# don't wait for specs to be generated (takes 10 sec or so)
with unittest.mock.patch("__main__.export_async_spec"):

    # mock up send method of AIOKafkaProducer
    with mock_AIOKafkaProducer_send() as mock:

        app = create_testing_app()

        @app.produces()
        async def to_my_test_topic(mobile: str, url: str) -> MyMsgUrl:
            msg = MyMsgUrl(info=dict(mobile=mobile, name="James Bond"), url=url)
            return msg

        @app.produces()
        def to_my_test_topic_2(mobile: str, url: str) -> MyMsgUrl:
            msg = MyMsgUrl(info=dict(mobile=mobile, name="James Bond"), url=url)
            return msg

        try:
            await app._on_startup()
            await to_my_test_topic(mobile="+385912345678", url="https://www.vip.hr")
            to_my_test_topic_2(mobile="+385987654321", url="https://www.ht.hr")
        finally:
            await app._on_shutdown()

        mock.assert_has_calls(
            [
                unittest.mock.call(
                    "my_test_topic",
                    b'{"info": {"mobile": "+385912345678", "name": "James Bond"}, "url": "https://www.vip.hr"}',
                ),
                unittest.mock.call(
                    "my_test_topic_2",
                    b'{"info": {"mobile": "+385987654321", "name": "James Bond"}, "url": "https://www.ht.hr"}',
                ),
            ]
        )

[INFO] __main__: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fast-kafka-api-kafka-1:9092'}'
[INFO] __main__: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fast-kafka-api-kafka-1:9092'}'
[INFO] fast_kafka_api._components.aiokafka_producer_manager: AIOKafkaProducerManager.start(): Entering...
[INFO] fast_kafka_api._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Starting...
[INFO] fast_kafka_api._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Starting task group
[INFO] fast_kafka_api._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Starting send_stream
[INFO] fast_kafka_api._components.aiokafka_producer_manager: AIOKafkaProducerManager.start(): Finished.
[INFO] fast_kafka_api._components.aiokafka_producer_manager: AIOKafkaProducerManager.stop(): Entering...
[INFO] fast_kafka_api._components.aiokafka_producer_manager: _aiokafka_producer_manager()

In [ ]:
# don't wait for specs to be generated (takes 10 sec or so)
with unittest.mock.patch("__main__.export_async_spec"):

    app = create_testing_app()
    fast_task = unittest.mock.Mock()
    long_task = unittest.mock.Mock()

    @app.run_in_background()
    async def bg_task():
        fast_task()
        await asyncio.sleep(100)
        long_task()

    fast_task_second = unittest.mock.Mock()
    long_task_second = unittest.mock.Mock()

    @app.run_in_background()
    async def bg_task_second():
        fast_task_second()
        await asyncio.sleep(100)
        long_task_second()

    try:
        await app._on_startup()
    finally:
        await app._on_shutdown()

    fast_task.assert_called()
    long_task.assert_not_called()

    fast_task_second.assert_called()
    long_task_second.assert_not_called()
    
print("ok")

ok
